In [2]:
!pip install -q pogema==1.2.2
!pip install -q stable_baselines3==2.1.0
!pip install -q sb3-contrib==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
import gymnasium as gym
from pogema import GridConfig

from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=30
)

env = gym.make("Pogema-v0",grid_config=grid_config)

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [4]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,


)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
!pip install -q optuna
!pip install -q optuna.pruners
!pip install -q optuna.samplers

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement optuna.pruners (from versions: none)
ERROR: No matching distribution found for optuna.pruners
ERROR: Could not find a version that satisfies the requirement optuna.samplers (from versions: none)
ERROR: No matching distribution found for optuna.samplers


In [6]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpLstmPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_recurrentppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for RecurrentPPO hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]


    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }

class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_recurrentppo_params(trial))
    # Create the RL model.
    model = RecurrentPPO(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2023-11-05 05:55:20,298] A new study created in memory with name: no-name-e9a2472d-3fb9-47b0-90ef-917c6c590338
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you s

流式输出内容被截断，只能显示最后 5000 行内容。
|    clip_range           | 0.2         |
|    entropy_loss         | -1.26       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0325      |
|    loss                 | 0.0178      |
|    n_updates            | 2310        |
|    policy_gradient_loss | -0.00315    |
|    value_loss           | 0.0691      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 49.7        |
|    ep_rew_mean          | 0.35        |
| time/                   |             |
|    fps                  | 19          |
|    iterations           | 233         |
|    time_elapsed         | 3012        |
|    total_timesteps      | 59648       |
| train/                  |             |
|    approx_kl            | 0.020347334 |
|    clip_fraction        | 0.1         |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.22       |
|    ex

[I 2023-11-05 07:36:29,619] Trial 0 finished with value: 0.0 and parameters: {'gamma': 0.029293958371423453, 'max_grad_norm': 4.532514282648331, 'gae_lambda': 0.010756515982743253, 'exponent_n_steps': 8, 'lr': 0.032505276465033796, 'ent_coef': 0.004931452581236477, 'ortho_init': False, 'net_arch': 'tiny', 'activation_fn': 'relu'}. Best is trial 0 with value: 0.0.


Number of finished trials:  1
Best trial:
  Value:  0.0
  Params: 
    gamma: 0.029293958371423453
    max_grad_norm: 4.532514282648331
    gae_lambda: 0.010756515982743253
    exponent_n_steps: 8
    lr: 0.032505276465033796
    ent_coef: 0.004931452581236477
    ortho_init: False
    net_arch: tiny
    activation_fn: relu
  User attrs:
    gamma_: 0.9707060416285765
    gae_lambda_: 0.9892434840172567
    n_steps: 256


In [10]:
recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,
    gamma=0.029293958371423453,
    learning_rate=0.032505276465033796,
    max_grad_norm=4.532514282648331,
    gae_lambda=0.010756515982743253,
    ent_coef=0.004931452581236477,
    n_steps=256,

)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
# Train agent and save it
recurrent_ppo.learn(total_timesteps=int(1.2e5))
recurrent_ppo.save("saved/recurrentppo_baseline")

流式输出内容被截断，只能显示最后 5000 行内容。
|    value_loss           | 0.0115    |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 27.5      |
|    ep_rew_mean          | 0.09      |
| time/                   |           |
|    fps                  | 18        |
|    iterations           | 232       |
|    time_elapsed         | 3137      |
|    total_timesteps      | 59392     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.49e-06 |
|    explained_variance   | 1.19e-07  |
|    learning_rate        | 0.0325    |
|    loss                 | 0.0029    |
|    n_updates            | 2310      |
|    policy_gradient_loss | 0.000251  |
|    value_loss           | 0.00812   |
---------------------------------------
---------------------------------------
| rollout/   

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'saved' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


Testing


In [12]:

from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=100):
    success_count = 0
    step_array = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 100
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                env.save_animation(f"render{i}.svg", AnimationConfig(egocentric_idx=0))
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(recurrent_ppo, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

流式输出内容被截断，只能显示最后 5000 行内容。
3 49 6 False
3 48 6 False
3 47 6 False
3 46 6 False
3 45 6 False
3 44 6 False
3 43 6 False
3 42 6 False
3 41 6 False
3 40 6 False
3 39 6 False
3 38 6 False
3 37 6 False
3 36 6 False
3 35 6 False
3 34 6 False
3 33 6 False
3 32 6 False
3 31 6 False
3 30 6 False
3 29 6 False
3 28 6 False
3 27 6 False
3 26 6 False
3 25 6 False
3 24 6 False
3 23 6 False
3 22 6 False
3 21 6 False
3 20 6 False
3 19 6 False
3 18 6 False
3 17 6 False
3 16 6 False
3 15 6 False
3 14 6 False
3 13 6 False
3 12 6 False
3 11 6 False
3 10 6 False
3 9 6 False
3 8 6 False
3 7 6 False
3 6 6 False
3 5 6 False
3 4 6 False
3 3 6 False
3 2 6 False
3 1 6 False
---51---
3 100 6 False
3 99 6 False
3 98 6 False
3 97 6 False
3 96 6 False
3 95 6 False
3 94 6 False
3 93 6 False
3 92 6 False
3 91 6 False
3 90 6 False
3 89 6 False
3 88 6 False
3 87 6 False
3 86 6 False
3 85 6 False
3 84 6 False
3 83 6 False
3 82 6 False
3 81 6 False
3 80 6 False
3 79 6 False
3 78 6 False
3 77 6 False
3 76 6 False
3 75 6 Fal